<a href="https://colab.research.google.com/github/chandresh564/Restaurant-rating-prediction/blob/main/Restaurant_rating_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading the dataset

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/zomato.csv')

In [3]:
df.sample(5)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
9176,https://www.zomato.com/bangalore/oh-my-biryani...,"47, Ground Floor - Nanda Nilaya, 18th Main Roa...",Oh My Biryani,Yes,No,3.7/5,439,+91 8197470002\r\n+91 9108318302,HSR,"Takeaway, Delivery","Raita, Butter Chicken Biryani, Shahi Tukda, Gu...","Biryani, North Indian",550,"[('Rated 4.0', 'RATED\n I ordered from this p...",[],Delivery,BTM
6465,https://www.zomato.com/bangalore/sri-sai-sagar...,"14/15, Manipal Center 47, Dickenson Road, MG R...",Sri Sai Sagar Food,No,No,NaN,0,+91 9845815667,MG Road,Quick Bites,NaN,"Street Food, Fast Food, North Indian",300,[],[],Dine-out,Brigade Road
12969,https://www.zomato.com/bangalore/sangam-kabab-...,"Opposite RBANMS Grounds, St John's Road, Ulsoo...",Sangam Kabab Corner,No,No,3.1/5,4,+91 9886069933,Ulsoor,Quick Bites,NaN,"Fast Food, North Indian",300,"[('Rated 1.0', ""RATED\n I tried this place be...",[],Dine-out,Church Street
39629,https://www.zomato.com/bangalore/by-2-coffee-m...,"49, 7th Cross, Malleshwaram, Bangalore",By 2 Coffee,Yes,No,4.1 /5,219,+91 9632203201,Malleshwaram,Quick Bites,"Filter Coffee, Kharabath, Masala Dosa, Tea, Id...",South Indian,100,"[('Rated 5.0', 'RATED\n Great place for coffe...",[],Delivery,Malleshwaram
33569,https://www.zomato.com/bangalore/bombay-kulfi-...,"45, Jakkasandra Cross, Koramangala 1st Block, ...",Bombay Kulfi,No,No,NaN,0,+91 9980934503,Koramangala 1st Block,Dessert Parlor,NaN,"Desserts, Ice Cream",150,[],[],Delivery,Koramangala 6th Block


In [4]:
df.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [9]:
df['listed_in(city)'].value_counts()

BTM                      3279
Koramangala 7th Block    2938
Koramangala 5th Block    2836
Koramangala 4th Block    2779
Koramangala 6th Block    2623
Jayanagar                2371
JP Nagar                 2096
Indiranagar              1860
Church Street            1827
MG Road                  1811
Brigade Road             1769
Lavelle Road             1744
HSR                      1741
Marathahalli             1659
Residency Road           1620
Whitefield               1620
Bannerghatta Road        1617
Brookefield              1518
Old Airport Road         1425
Kammanahalli             1329
Kalyan Nagar             1309
Basavanagudi             1266
Sarjapur Road            1261
Electronic City          1229
Bellandur                1227
Frazer Town              1185
Malleshwaram             1096
Rajajinagar              1079
Banashankari              863
New BEL Road              740
Name: listed_in(city), dtype: int64

In [10]:
df['location'].value_counts()

BTM                      5124
HSR                      2523
Koramangala 5th Block    2504
JP Nagar                 2235
Whitefield               2144
                         ... 
West Bangalore              6
Yelahanka                   6
Jakkur                      3
Rajarajeshwari Nagar        2
Peenya                      1
Name: location, Length: 93, dtype: int64

# Dropping unnecessary columns

In [7]:
# menu_item,reviews list,dish liked columns have too many empty values which provide no information. So dropping them
# city and location are related. Location cannot be choosen as a feature as it has too many unique values
df_copy=df.drop(['url','address','phone', 'reviews_list','menu_item','dish_liked','location','name'],axis=1)

# Renaming the columns

In [11]:
df_copy=df_copy.rename(columns={'listed_in(city)':'location',
                                'rate':'rating',
                                'rest_type':'type',
                                'listed_in(type)':'listed_type',
                                'approx_cost(for two people)':'cost'
                                })

# Feature Engineering

In [12]:
# 1. votes column
# dropping restaurants with no votes
df_copy=df_copy[df_copy['votes']!=0]
df_copy = df_copy.reset_index(drop=True)

In [13]:
df_copy

,online_order,book_table,rating,votes,type,cuisines,cost,listed_type,location
0,Yes,Yes,4.1/5,775,Casual Dining,"North Indian, Mughlai, Chinese",800,Buffet,Banashankari
1,Yes,No,4.1/5,787,Casual Dining,"Chinese, North Indian, Thai",800,Buffet,Banashankari
2,Yes,No,3.8/5,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,Buffet,Banashankari
3,No,No,3.7/5,88,Quick Bites,"South Indian, North Indian",300,Buffet,Banashankari
4,No,No,3.8/5,166,Casual Dining,"North Indian, Rajasthani",600,Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...
41685,No,No,3.7 /5,34,"Casual Dining, Bar","North Indian, Continental",800,Pubs and bars,Whitefield
41686,No,No,2.5 /5,81,"Casual Dining, Bar","Andhra, South Indian, Chinese, North Indian",800,Pubs and bars,Whitefield
41687,No,No,3.6 /5,27,Bar,Continental,"1,500",Pubs and bars,Whitefield
41688,No,Yes,4.3 /5,236,Bar,Finger Food,"2,500",Pubs and bars,Whitefield


In [14]:
# 2. rating column
# converting rating of the form x/5 to x and dropping restaurants with no rating
import math
def convert(value):
    val = str(value).split('/')
    return round(float(val[0]),1)
df_copy['rating'] = df_copy['rating'].apply(convert)
df_copy = df_copy[df_copy['rating'].apply(lambda x: not math.isnan(x))]

In [15]:
df_copy

,online_order,book_table,rating,votes,type,cuisines,cost,listed_type,location
0,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800,Buffet,Banashankari
1,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800,Buffet,Banashankari
2,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,Buffet,Banashankari
3,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300,Buffet,Banashankari
4,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600,Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...
41685,No,No,3.7,34,"Casual Dining, Bar","North Indian, Continental",800,Pubs and bars,Whitefield
41686,No,No,2.5,81,"Casual Dining, Bar","Andhra, South Indian, Chinese, North Indian",800,Pubs and bars,Whitefield
41687,No,No,3.6,27,Bar,Continental,"1,500",Pubs and bars,Whitefield
41688,No,Yes,4.3,236,Bar,Finger Food,"2,500",Pubs and bars,Whitefield


In [ ]:
# 3. cost column
# replacing the missing values in cost_for_two column by mean of the groupby objects
df_copy['cost'] = pd.to_numeric(df_copy['cost'], errors='coerce').astype('Float64')
df_copy['cost'] = df_copy.groupby(['location', 'listed_type'])['cost'].transform(lambda x: x.fillna(x.mean()))
df_copy['cost'] = df_copy.groupby(['location'])['cost'].transform(lambda x: x.fillna(x.mean()))

In [17]:
df_copy

,online_order,book_table,rating,votes,type,cuisines,cost,listed_type,location
0,Yes,Yes,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
1,Yes,No,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet,Banashankari
2,Yes,No,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
3,No,No,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet,Banashankari
4,No,No,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...
41685,No,No,3.7,34,"Casual Dining, Bar","North Indian, Continental",800.0,Pubs and bars,Whitefield
41686,No,No,2.5,81,"Casual Dining, Bar","Andhra, South Indian, Chinese, North Indian",800.0,Pubs and bars,Whitefield
41687,No,No,3.6,27,Bar,Continental,800.0,Pubs and bars,Whitefield
41688,No,Yes,4.3,236,Bar,Finger Food,800.0,Pubs and bars,Whitefield


In [ ]:
# 4. cuisines/dishes column
# each cuisne offered at atleast 100 restaurants is taken as a feature

df_copy.dropna(subset='cuisines',inplace=True)
df_copy = df_copy.reset_index(drop=True)

dic={}
for i in df_copy['cuisines']:
  for j in i.split(','):
    if j.lstrip() in dic:
      dic[j.lstrip()]+=1
    else:
      dic[j.lstrip()]=1

cuisines_to_keep=[]
for i,j in dic.items():
  if j>100:
    cuisines_to_keep.append(i)

df_copy['cuisines']=df_copy['cuisines'].apply(lambda x: [value.lstrip() for value in x.split(',')])

In [19]:
df_copy

,online_order,book_table,rating,votes,type,cuisines,cost,listed_type,location
0,Yes,Yes,4.1,775,Casual Dining,"[North Indian, Mughlai, Chinese]",800.0,Buffet,Banashankari
1,Yes,No,4.1,787,Casual Dining,"[Chinese, North Indian, Thai]",800.0,Buffet,Banashankari
2,Yes,No,3.8,918,"Cafe, Casual Dining","[Cafe, Mexican, Italian]",800.0,Buffet,Banashankari
3,No,No,3.7,88,Quick Bites,"[South Indian, North Indian]",300.0,Buffet,Banashankari
4,No,No,3.8,166,Casual Dining,"[North Indian, Rajasthani]",600.0,Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...
41630,No,No,3.7,34,"Casual Dining, Bar","[North Indian, Continental]",800.0,Pubs and bars,Whitefield
41631,No,No,2.5,81,"Casual Dining, Bar","[Andhra, South Indian, Chinese, North Indian]",800.0,Pubs and bars,Whitefield
41632,No,No,3.6,27,Bar,[Continental],800.0,Pubs and bars,Whitefield
41633,No,Yes,4.3,236,Bar,[Finger Food],800.0,Pubs and bars,Whitefield


In [23]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
encoded_features = mlb.fit_transform(df_copy['cuisines'])
one_hot_encoded = pd.DataFrame(encoded_features, columns=mlb.classes_)
one_hot_encoded

,Afghan,Afghani,African,American,Andhra,Arabian,Asian,Assamese,Australian,Awadhi,...,Sushi,Tamil,Tea,Tex-Mex,Thai,Tibetan,Turkish,Vegan,Vietnamese,Wraps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41631,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41632,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41633,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_combined = pd.concat([df_copy, one_hot_encoded[cuisines_to_keep]], axis=1)

df_combined = df_combined.loc[~(df_combined[cuisines_to_keep] == 0).all(axis=1)]

df_combined = df_combined.drop(columns=['cuisines']).reset_index(drop=True)

In [25]:
df_combined

,online_order,book_table,rating,votes,type,cost,listed_type,location,North Indian,Mughlai,...,Lebanese,Finger Food,Tibetan,Mediterranean,Kerala,Oriya,Maharashtrian,Chettinad,Modern Indian,Malaysian
0,Yes,Yes,4.1,775,Casual Dining,800.0,Buffet,Banashankari,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Yes,No,4.1,787,Casual Dining,800.0,Buffet,Banashankari,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Yes,No,3.8,918,"Cafe, Casual Dining",800.0,Buffet,Banashankari,0,0,...,0,0,0,0,0,0,0,0,0,0
3,No,No,3.7,88,Quick Bites,300.0,Buffet,Banashankari,1,0,...,0,0,0,0,0,0,0,0,0,0
4,No,No,3.8,166,Casual Dining,600.0,Buffet,Banashankari,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41536,No,No,3.7,34,"Casual Dining, Bar",800.0,Pubs and bars,Whitefield,1,0,...,0,0,0,0,0,0,0,0,0,0
41537,No,No,2.5,81,"Casual Dining, Bar",800.0,Pubs and bars,Whitefield,1,0,...,0,0,0,0,0,0,0,0,0,0
41538,No,No,3.6,27,Bar,800.0,Pubs and bars,Whitefield,0,0,...,0,0,0,0,0,0,0,0,0,0
41539,No,Yes,4.3,236,Bar,800.0,Pubs and bars,Whitefield,0,0,...,0,1,0,0,0,0,0,0,0,0


In [26]:
# 5. listed_type column
# can be changed
df_combined.drop(columns=['listed_type'],inplace=True)

In [27]:
# 6. type column
# similar to cuisines column. The type column is one hot encoded with each type as a feature

df_combined.dropna(subset='type',inplace=True)
df_combined = df_combined.reset_index(drop=True)

df_combined['type']=df_combined['type'].apply(lambda x: [value.lstrip() for value in x.split(',')])

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

one_hot_encoded = pd.DataFrame(mlb.fit_transform(df_combined['type']), columns=mlb.classes_)

df_final = pd.concat([df_combined, one_hot_encoded], axis=1)

df_final = df_final.drop(columns=['type'])

In [28]:
# 7. location column
# location column is also one hot encoded

df_final = pd.get_dummies(df_final, columns=['location'], prefix='', prefix_sep='')

In [29]:
# 8. online order and book table columns

df_final['online_order'] = df_final['online_order'].map({'Yes': 1, 'No': 0})
df_final['book_table'] = df_final['book_table'].map({'Yes': 1, 'No': 0})

# Final columns

In [30]:
df_final.columns

Index(['online_order', 'book_table', 'rating', 'votes', 'cost', 'North Indian',
       'Mughlai', 'Chinese', 'Thai', 'Cafe',
       ...
       'Lavelle Road', 'MG Road', 'Malleshwaram', 'Marathahalli',
       'New BEL Road', 'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'],
      dtype='object', length=113)

# Standard Scaling

In [31]:
from sklearn.preprocessing import StandardScaler
cost_values = df_final['cost'].values.reshape(-1, 1)
scaler = StandardScaler()
df_final['cost'] = scaler.fit_transform(cost_values)
df_final

,online_order,book_table,rating,votes,cost,North Indian,Mughlai,Chinese,Thai,Cafe,...,Lavelle Road,MG Road,Malleshwaram,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield
0,1,1,4.1,775,1.782893,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,4.1,787,1.782893,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,3.8,918,1.782893,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,3.7,88,-0.810045,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,3.8,166,0.745718,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41393,0,0,3.7,34,1.782893,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41394,0,0,2.5,81,1.782893,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
41395,0,0,3.6,27,1.782893,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41396,0,1,4.3,236,1.782893,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Train-Test split

In [62]:
from sklearn.model_selection import train_test_split
y=df_final['rating']
X=df_final.drop(['rating'],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Model selection and training

In [63]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(oob_score=True)

In [64]:
rfr.fit(X_train,y_train)

RandomForestRegressor(oob_score=True)

In [65]:
from sklearn.metrics import r2_score
y_preds = rfr.predict(X_test)

print(r2_score(y_preds, y_test))

0.8710594044496133


In [36]:
print(rfr.score(X_train,y_train))
print(rfr.score(X_test,y_test))

0.9831327948514808
0.8964754665933272


In [46]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators':[50,100,200],
              'min_samples_split':[5,10,20],
              'max_features':[0.4,0.6,1],
              }

estimator = RandomForestRegressor()

reg = GridSearchCV(estimator=estimator, param_grid=param_grid,cv=10,scoring='r2',verbose=1)

reg.fit(X_train,y_train)

Fitting 10 folds for each of 27 candidates, totalling 270 fits


GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_features': [0.4, 0.6, 1],
                         'min_samples_split': [5, 10, 20],
                         'n_estimators': [50, 100, 200]},
             scoring='r2', verbose=1)

In [47]:
reg.best_params_

{'max_features': 0.6, 'min_samples_split': 5, 'n_estimators': 200}

In [48]:
reg.best_score_

0.8647971068233569

In [50]:
# checking the accuracy by dropping votes column

from sklearn.model_selection import train_test_split
y=df_final['rating']
X=df_final.drop(['rating','votes'],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [52]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(oob_score=True)
rfr.fit(X_train,y_train)

RandomForestRegressor(oob_score=True)

In [53]:
print(rfr.score(X_train,y_train))
print(rfr.score(X_test,y_test))

0.8994508396761556
0.7010712525714798


In [57]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()

gbr.fit(X_train,y_train)

GradientBoostingRegressor()

In [58]:
print(gbr.score(X_train,y_train))
print(gbr.score(X_test,y_test))

0.5328333243475527
0.535061101336888
